Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [11]:
import os

WORKSPACE_PATH = r'Tensorflow\workspace'
SCRIPTS_PATH = r'Tensorflow\scripts'
APIMODEL_PATH = r'Tensorflow\models'
ANNOTATION_PATH = WORKSPACE_PATH + r'\annotations'
IMAGE_PATH = WORKSPACE_PATH + r'\images'
MODEL_PATH = WORKSPACE_PATH + r'\models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + r'\pre-trained-models'
CONFIG_PATH = MODEL_PATH + r'\my_ssd_mobnet\pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + r'\my_ssd_mobnet\\'

generate_tfrecord = os.path.join(SCRIPTS_PATH, 'generate_tfrecord.py')
train_image_path = os.path.join(IMAGE_PATH, 'train')
test_image_path = os.path.join(IMAGE_PATH, 'test')
label_map = os.path.join(ANNOTATION_PATH, 'label_map.pbtxt')
train_record = os.path.join(ANNOTATION_PATH, 'train.record')
test_record = os.path.join(ANNOTATION_PATH, 'test.record')

# 1. Create Label Map

In [2]:
labels = [
    {'name':'Hello', 'id':1}, 
    {'name':'Yes', 'id':2},
    {'name':'No', 'id':3},
    {'name':'Thank You', 'id':4},
    {'name':'I Love You', 'id':5},
    {'name':'Peace Among Worlds', 'id':6}
    ]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF (tensorflow) records

In [12]:
!python {generate_tfrecord} -x {train_image_path} -l {label_map} -o {train_record}
!python {generate_tfrecord} -x {test_image_path} -l {label_map} -o {test_record}

Traceback (most recent call last):
  File "c:\Users\Cr4sZ\Github\Sign-Language-Detector\Tensorflow\scripts\generate_tfrecord.py", line 29, in <module>
    from object_detection.utils import dataset_util, label_map_util
  File "c:\Users\Cr4sZ\anaconda3\Lib\site-packages\object_detection\utils\label_map_util.py", line 21, in <module>
    from object_detection.protos import string_int_label_map_pb2
ImportError: cannot import name 'string_int_label_map_pb2' from 'object_detection.protos' (c:\Users\Cr4sZ\anaconda3\Lib\site-packages\object_detection\protos\__init__.py)
Traceback (most recent call last):
  File "c:\Users\Cr4sZ\Github\Sign-Language-Detector\Tensorflow\scripts\generate_tfrecord.py", line 29, in <module>
    from object_detection.utils import dataset_util, label_map_util
  File "c:\Users\Cr4sZ\anaconda3\Lib\site-packages\object_detection\utils\label_map_util.py", line 21, in <module>
    from object_detection.protos import string_int_label_map_pb2
ImportError: cannot import name

In [ ]:
import os

generate_tfrecord = os.path.join(SCRIPTS_PATH, 'generate_tfrecord.py')
train_image_path = os.path.join(IMAGE_PATH, 'train')
test_image_path = os.path.join(IMAGE_PATH, 'test')
label_map = os.path.join(ANNOTATION_PATH, 'label_map.pbtxt')
train_record = os.path.join(ANNOTATION_PATH, 'train.record')
test_record = os.path.join(ANNOTATION_PATH, 'test.record')

!python {generate_tfrecord} -x {train_image_path} -l {label_map} -o {train_record}
!python {generate_tfrecord} -x {test_image_path} -l {label_map} -o {test_record}

In [7]:
%pip install object-detection

  Obtaining dependency information for object-detection from https://files.pythonhosted.org/packages/b7/37/24d57adb29a0242aca08b99cb2ad08c2b427c05c988f169622c401a6f9ed/object_detection-0.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for contextlib2 from https://files.pythonhosted.org/packages/76/56/6d6872f79d14c0cb02f1646cbb4592eef935857c0951a105874b7b62a0c3/contextlib2-21.6.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.5 MB 2.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 9.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 9.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 9.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 9.4 MB/s eta 0:00:01
   ---------------------------- ----------- 1.

# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [4]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...


In [6]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [7]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [47]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


# 5. Update Config For Transfer Learning

In [1]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [8]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [55]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [56]:
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
    

In [52]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [53]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [54]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [58]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000


# 7. Load Train Model From Checkpoint

In [2]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [9]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [10]:
import cv2 
import numpy as np

In [11]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [105]:
cap.release()

In [12]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

In [42]:
detections = detect_fn(input_tensor)

In [67]:
from matplotlib import pyplot as plt